<a href="https://colab.research.google.com/github/MuafiraThasni/Airline-Price-Optimization/blob/main/price_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tqdm import tqdm
from IPython.display import display, Javascript
import json
from collections import namedtuple
from numpy.random import uniform, seed
from numpy import floor
import matplotlib.pyplot as plt

In [2]:
def _tickets_sold(p, demand_level, max_qty):
    quantity_demanded = floor(max(0, p - demand_level))
    return min(quantity_demanded, max_qty)

In [3]:
# Simulate revenues achieved by pricing function on a event (days_left before event, number of tickets to sell) with random demand

def simulate_revenue(days_left, tickets_left, pricing_function, rev_to_date=0, demand_level_min=100, demand_level_max=200, verbose=False):
    
    if (days_left == 0) or (tickets_left == 0):
        
        if verbose:
            if (days_left == 0):
                print("The flight took off today. ")
            if (tickets_left == 0):
                print("This flight is booked full.")
            print("Total Revenue: €{:.0f}".format(rev_to_date))
        return rev_to_date
    
    else:
        
        demand_level = uniform(demand_level_min, demand_level_max)
        p = pricing_function(days_left, tickets_left, demand_level)
        q = _tickets_sold(demand_level, p, tickets_left)
        
        if verbose:
            print("{:.0f} days before flight: "
                  "Started with {:.0f} seats. "
                  "Demand level: {:.0f}. "
                  "Price set to €{:.0f}. "
                  "Sold {:.0f} tickets. "
                  "Daily revenue is {:.0f}. Total revenue-to-date is {:.0f}. "
                  "{:.0f} seats remaining".format(days_left, tickets_left, demand_level, p, q, p*q, p*q+rev_to_date, tickets_left-q))
        return simulate_revenue(days_left = days_left-1,
                              tickets_left = tickets_left-q,
                              pricing_function=pricing_function,
                              rev_to_date=rev_to_date + p * q,
                              demand_level_min=demand_level_min,
                              demand_level_max=demand_level_max,
                              verbose=verbose)

In [4]:
# Calculate average revenues per event achieved by pricing function on a range of events => To compare algorithms

def score_me(pricing_function, sims_per_scenario=200):
    
    seed(0)
    Scenario = namedtuple('Scenario', 'n_days n_tickets')
    scenarios = [Scenario(n_days=100, n_tickets=100), Scenario(n_days=14, n_tickets=50),
                 Scenario(n_days=2, n_tickets=20), Scenario(n_days=1, n_tickets=3),
                 ]
    scenario_scores = []
    
    for s in scenarios:
        
        scenario_score = sum(simulate_revenue(s.n_days, s.n_tickets, pricing_function)
                                     for _ in range(sims_per_scenario)) / sims_per_scenario
        
        print("Ran {:.0f} flights starting {:.0f} days before flight with {:.0f} tickets. "
              "Average revenue: €{:.0f}".format(sims_per_scenario, s.n_days, s.n_tickets, scenario_score))
        
        scenario_scores.append(scenario_score)
    
    score = sum(scenario_scores) / len(scenario_scores)
    
    print("Average revenue across all flights is €{:.0f}".format(score))

In [5]:
def basic_pricing_function(days_left, tickets_left, demand_level):
    """Sample pricing function"""
    price = demand_level - 10
    return price

In [6]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=basic_pricing_function, verbose=True)
print()

7 days before flight: Started with 50 seats. Demand level: 192. Price set to €182. Sold 10 tickets. Daily revenue is 1818. Total revenue-to-date is 1818. 40 seats remaining
6 days before flight: Started with 40 seats. Demand level: 160. Price set to €150. Sold 10 tickets. Daily revenue is 1500. Total revenue-to-date is 3318. 30 seats remaining
5 days before flight: Started with 30 seats. Demand level: 154. Price set to €144. Sold 10 tickets. Daily revenue is 1436. Total revenue-to-date is 4754. 20 seats remaining
4 days before flight: Started with 20 seats. Demand level: 185. Price set to €175. Sold 10 tickets. Daily revenue is 1755. Total revenue-to-date is 6509. 10 seats remaining
3 days before flight: Started with 10 seats. Demand level: 119. Price set to €109. Sold 10 tickets. Daily revenue is 1091. Total revenue-to-date is 7599. 0 seats remaining
This flight is booked full.
Total Revenue: €7599



In [7]:
# we assume max day left = 100, max tickets to sell = 100
n = 100
optimal_prices = np.zeros((n+1, n+1, n+1), dtype='uint8') # The optimal price given the number of days left (0->100), tickets left (0->100) and current demand level (100->200)
optimal_sales = np.zeros((n+1, n+1), dtype=np.float)      # Expected profit when pricing optimally given the number of days and tickets left

# Base case when there is 1 day left
for tickets_left in range(n+1):
    
    for demand in range(n, 2*n+1):
        
        tickets_sold = min(demand // 2, tickets_left) # demand / 2 is the optimal quantity to sell if there are enough tickets left
        price = demand - tickets_sold
        
        optimal_prices[1, tickets_left, demand - 100] = price
        
        # Calculate optimal revenue expectation. Sum over every demand level with equal chance of occurence (uniform distribution)
        optimal_sales[1, tickets_left] += (price * tickets_sold / 101) # Average out the best profit through all possible demand levels
        
        # Fill in the tables
for days_left in tqdm(range(2, n+1)):
    
    for tickets_left in range(n+1):
        
        last_best_price = 0 # For previous demand level (one lower)
        
        for demand in range(n, 2*n+1):
            
            # Search over all prices to find the best price for the context: days_left, tickets_left, demand_level
            best_revenue = 0
            best_price = 0
                        
            # No need to search all prices in range(0, demand):
            for price in range(max(demand - tickets_left, last_best_price), demand+1): # optimal price should not be lower than demand - remaining tickets or the last best price
                
                tickets_sold = min(demand - price, tickets_left) # This is the quantity that will be sold at price p
                revenue = price * tickets_sold + optimal_sales[days_left - 1, tickets_left - tickets_sold] # This is the total expected profit if selling at price p
                
                if revenue > best_revenue: # Update best profit and price
                    best_revenue, best_price = revenue, price
            
            # Set the optimal price and profit values
            optimal_prices[days_left, tickets_left, demand-100] = best_price
            
            # similarly let's calculate the optimal_sales expectation over all possible demand level (same probability of occurence)
            optimal_sales[days_left, tickets_left] += (best_revenue / 101) # Average out the best profit through all possible demand level
            
            last_best_price = best_price # For higher demand levels, this best_price is the floor

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.
100%|██████████| 99/99 [00:06<00:00, 14.83it/s]


In [8]:
def pricing_function_bf(days_left, tickets_left, demand_level):
    """Return the optimal price"""
    
    # These should be cast to an integer (and rounded down)
    tickets_left = int(tickets_left)
    demand_level = int(demand_level)
    
    return optimal_prices[days_left, tickets_left, demand_level-100] # Return the precomputed values

In [9]:
print('pricing function:')

simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function_bf, verbose=True)
score_me(pricing_function_bf, 200)

pricing function:
7 days before flight: Started with 50 seats. Demand level: 121. Price set to €121. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 0. 50 seats remaining
6 days before flight: Started with 50 seats. Demand level: 139. Price set to €136. Sold 2 tickets. Daily revenue is 272. Total revenue-to-date is 272. 48 seats remaining
5 days before flight: Started with 48 seats. Demand level: 110. Price set to €109. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 272. 48 seats remaining
4 days before flight: Started with 48 seats. Demand level: 108. Price set to €108. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 272. 48 seats remaining
3 days before flight: Started with 48 seats. Demand level: 114. Price set to €109. Sold 4 tickets. Daily revenue is 436. Total revenue-to-date is 708. 44 seats remaining
2 days before flight: Started with 44 seats. Demand level: 106. Price set to €94. Sold 11 tickets. Daily revenue is 1034. Total revenue-to-d

In [10]:
from math import floor

savedF = {}
savedG = {}                                  

def getF(tickets,days):
    if tickets<=0 or days <= 0:
        return 0
    if (tickets,days) in savedF:
        return savedF[(tickets,days)]
    res = 0.0
    #count math expectation 
    for i in range(100, 201):
        res += getG(tickets, days, i)[0]
    res /= 101
    
    savedF[(tickets,days)] = res
    return res

def getG(tickets, days, demand):
    if days <= 0 or tickets <= 0:
        return (0,0)
    if (tickets, days, demand) in savedG:
        return savedG[(tickets, days, demand)]
    topTotal = -1
    topPrice = -1
    for i in range(1, demand+1):
        res = min(demand-i, tickets)*i + (getF(tickets-(demand-i), days-1) if tickets-(demand-i) > 0 else 0)
        if res > topTotal:
            topTotal, topPrice = res, i
    savedG[(tickets, days, demand)] = (topTotal, topPrice)
    return (topTotal, topPrice)

In [11]:
def pricing_function_dp(days_left, tickets_left, demand_level):
    demand_level_floor = int(floor(demand_level))
    demand_delta = demand_level - demand_level_floor
    (topTotal, topPrice) = getG(tickets_left, days_left, demand_level_floor)
    return topPrice + demand_delta - 0.00001

In [12]:
print('pricing function:')
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function_dp, verbose=True)
score_me(pricing_function_dp, 200)

pricing function:
7 days before flight: Started with 50 seats. Demand level: 198. Price set to €175. Sold 23 tickets. Daily revenue is 4028. Total revenue-to-date is 4028. 27 seats remaining
6 days before flight: Started with 27 seats. Demand level: 152. Price set to €151. Sold 1 tickets. Daily revenue is 151. Total revenue-to-date is 4179. 26 seats remaining
5 days before flight: Started with 26 seats. Demand level: 149. Price set to €147. Sold 2 tickets. Daily revenue is 293. Total revenue-to-date is 4472. 24 seats remaining
4 days before flight: Started with 24 seats. Demand level: 160. Price set to €153. Sold 7 tickets. Daily revenue is 1068. Total revenue-to-date is 5540. 17 seats remaining
3 days before flight: Started with 17 seats. Demand level: 109. Price set to €109. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 5540. 17 seats remaining
2 days before flight: Started with 17 seats. Demand level: 131. Price set to €127. Sold 4 tickets. Daily revenue is 507. Total

In [14]:
def create_valuefunctions(remaining_days,remaining_tickets,min_demand_level,max_demand_level):
    demand_levels_n = max_demand_level - min_demand_level
    Q = np.zeros([remaining_days + 1,remaining_tickets + 1,demand_levels_n,remaining_tickets + 1])
    V = np.zeros([remaining_days + 1,remaining_tickets + 1])
    return Q,V

def make_base_step(Q,V,demand_range,remaining_tickets):
    for tickets_left in range(1,remaining_tickets + 1):
        for demand_level_i,demand_level  in enumerate(demand_range):
            for tickets_sold in range(1, tickets_left + 1):
                price = demand_level - tickets_sold
                immediate_reward = tickets_sold * price
                Q[1,tickets_left,demand_level_i,tickets_sold] = immediate_reward
                
        V[1,tickets_left] = Q[1,tickets_left,:,:].max(axis = 1).mean()
        


In [15]:
def dynamic_programming_algorithm(Q,V,remaining_days,remaining_tickets,demand_range):
    for days_left in range(2, remaining_days +1):
        for tickets_left in range(1,remaining_tickets+1):
            for demand_level_i,demand_level  in enumerate(demand_range):
                for tickets_sold in range(1, tickets_left + 1):
                    price = demand_level - tickets_sold
                    immediate_reward = tickets_sold * price
                    Q[days_left,tickets_left,demand_level_i,tickets_sold] = immediate_reward + V[days_left-1,tickets_left - tickets_sold]
                
                V[days_left,tickets_left] = Q[days_left,tickets_left,:,:].max(axis = 1).mean()
    
    return Q,V

In [ ]:
remaining_days = 200
remaining_tickets = 150
min_demand_level = 100
max_demand_level = 200

demand_levels_n = max_demand_level - min_demand_level
demand_range = np.linspace(min_demand_level, max_demand_level,demand_levels_n , dtype = int)

Q,V = create_valuefunctions(remaining_days, remaining_tickets, min_demand_level, max_demand_level)
make_base_step(Q,V,demand_range,remaining_tickets)
Q,V = dynamic_programming_algorithm(Q,V,remaining_days,remaining_tickets,demand_range)

In [ ]:
def pricing_function_bm(days_left, tickets_left, demand_level):
    """Sample pricing function"""
    demand_level_index = np.abs(demand_level - demand_range).argmin()
    demand_level_index = demand_level_index
    relevant_tickest_sold = Q[days_left, int(tickets_left), demand_level_index, :]
    best_tickets_sold = relevant_tickest_sold.argmax()
    price = max(demand_level - best_tickets_sold,0)
    return price

In [ ]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function_bm, verbose=True)